In [39]:
!pip3 install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 1.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.1/39.1 MB 2.3 MB/s eta 0:00:0000:0100:01


In [60]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the dataset
raw_data = pd.read_csv('winequality-white.csv', sep='\t', header=None)  # Using tab as separator

#Display the first few rows to inspect the data
#print("Original DataFrame shape:", df.shape)
#print(df.head())

# Split the first column (which contains all attributes) by semicolons
df_split = raw_data[0].str.split(';', expand=True)

# Strip quotes from each attribute
df = df_split.apply(lambda x: x.str.replace('"', ''))

# Check the resulting DataFrame
print("Split DataFrame shape:", df.shape)
print(df)

Split DataFrame shape: (4899, 12)
                 0                 1            2               3          4   \
0     fixed acidity  volatile acidity  citric acid  residual sugar  chlorides   
1                 7              0.27         0.36            20.7      0.045   
2               6.3               0.3         0.34             1.6      0.049   
3               8.1              0.28          0.4             6.9       0.05   
4               7.2              0.23         0.32             8.5      0.058   
...             ...               ...          ...             ...        ...   
4894            6.2              0.21         0.29             1.6      0.039   
4895            6.6              0.32         0.36               8      0.047   
4896            6.5              0.24         0.19             1.2      0.041   
4897            5.5              0.29          0.3             1.1      0.022   
4898              6              0.21         0.38             0.8       0.

In [64]:
# Assuming the last column is the target variable
# X is the features, Y is the target
X = df.iloc[:,:-1]   # All columns except the last one
Y = df.iloc[:, -1]   # The last column is the target
#print(Y)
# Check the split data
print("Feature Matrix (X) shape:", X.shape)
print("Target Vector (Y) shape:", Y.shape)

Feature Matrix (X) shape: (4899, 11)
Target Vector (Y) shape: (4899,)


In [65]:
# Define your percentages
train = 0.7  # 70% for training
# validation = 0.15 # 15% for validation
# The remaining percentage for testing will be (100 - t - v)% = 15%

# Proceed with splitting the dataset
X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, test_size = (1-t), random_state = 1)
X_valid, X_test, Y_valid, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state= 1)

# Now you can use X_train, Y_train, etc.
print(X_train)  # To confirm the data is correctly split

       0      1     2     3      4   5    6        7     8     9     10
2150  6.8   0.24  0.34   5.1  0.038  31   99   0.9921  3.24  0.46  11.8
4898    6   0.21  0.38   0.8   0.02  22   98  0.98941  3.26  0.32  11.8
1586  7.3    0.3  0.74  13.5  0.039  46  165   0.9982  3.02   0.4   8.7
1518  8.2   0.24  0.49   9.3  0.038  52  163   0.9952  3.02  0.33  10.6
432   6.4   0.34   0.1   1.1  0.048  19   84   0.9927  3.21  0.38   9.8
...   ...    ...   ...   ...    ...  ..  ...      ...   ...   ...   ...
2895  6.4  0.125  0.36   1.4  0.044  22   68  0.99014  3.15   0.5  11.7
2763  7.3   0.32  0.35   1.4   0.05   8  163  0.99244  3.24  0.42  10.7
905   6.9   0.21  0.28   2.4  0.056  49  159   0.9944  3.02  0.47   8.8
3980  6.5   0.29  0.52   1.7  0.034  41  193  0.99223  3.25  0.59  10.4
235   7.2   0.23  0.38  14.3  0.058  55  194   0.9979  3.09  0.44     9

[3429 rows x 11 columns]
